## step 1: Read the csv file and convert it to a feature class in GIS


In [1]:
import csv
file = open('boundary.csv')
csv_reader = csv.reader(file)
for line in csv_reader:
    print(line)



['col', 'row', 'X', 'Y']
['4871', '174', '699102.8877924071', '186780.44581266836']
['4871', '174', '699102.8877924071', '186780.44581266836']
['4872', '174', '699105.8874190656', '186780.44581266836']
['4870', '175', '699099.8881657487', '186777.44618600988']
['4873', '174', '699108.8870457241', '186780.44581266836']
['4869', '175', '699096.8885390902', '186777.44618600988']
['4874', '174', '699111.8866723826', '186780.44581266836']
['4868', '175', '699093.8889124317', '186777.44618600988']
['4868', '175', '699093.8889124317', '186777.44618600988']
['4875', '174', '699114.886299041', '186780.44581266836']
['4876', '174', '699117.8859256995', '186780.44581266836']
['4876', '174', '699117.8859256995', '186780.44581266836']
['4867', '176', '699090.8892857733', '186774.44655935143']
['4876', '175', '699117.8859256995', '186777.44618600988']
['4866', '176', '699087.8896591148', '186774.44655935143']
['4865', '176', '699084.8900324563', '186774.44655935143']
['4877', '176', '699120.88555235

In [2]:
import arcpy
arcpy.da.Describe('flood_2class.tif')

{'catalogPath': 'flood_2class.tif',
 'FIDSet': None,
 'size': 1089313,
 'dateCreated': '2024-12-01T22:29:57.000',
 'dateAccessed': '2024-12-01T22:31:08.000',
 'dateModified': '2024-12-01T09:50:14.000',
 'workspace': <geoprocessing describe data object at 0x18eda160d70>,
 'supportsBigObjectID': False,
 'supportsBigInteger': False,
 'supportsTimeOnly': False,
 'supportsDateOnly': False,
 'supportsTimestampOffset': False,
 'bandCount': 1,
 'baseName': 'flood_2class',
 'canVersion': False,
 'changeTracked': False,
 'children': [{'catalogPath': 'flood_2class.tif\\Band_1',
   'FIDSet': None,
   'workspace': <geoprocessing describe data object at 0x18ee443e0b0>,
   'supportsBigObjectID': False,
   'supportsBigInteger': False,
   'supportsTimeOnly': False,
   'supportsDateOnly': False,
   'supportsTimestampOffset': False,
   'baseName': 'Band_1',
   'canVersion': False,
   'changeTracked': False,
   'children': [],
   'childrenExpanded': True,
   'dataElementType': 'DERasterBand',
   'datasetT

In [3]:
import arcpy
desc = arcpy.da.Describe('flood_2class.tif')
sr = desc['spatialReference']
sr

name (Projected Coordinate System),NAD_1983_StatePlane_North_Carolina_FIPS_3200
factoryCode (WKID),32119
linearUnitName (Linear Unit),Meter
name (Geographic Coordinate System),GCS_North_American_1983
factoryCode (WKID),4269
angularUnitName (Angular Unit),Degree
datumName (Datum),D_North_American_1983


In [14]:
arcpy.env.workspace = r'D:\project2'
import os
input=os.path.join(arcpy.env.workspace,'boundary.csv')
out = os.path.join(arcpy.env.workspace, 'boundary_pnts.shp')
arcpy.management.XYTableToPoint(in_table=input, out_feature_class=out,x_field='X',y_field='Y', coordinate_system=32119)


<Result 'D:\\project2\\boundary_pnts.shp'>

In [18]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Load the CSV file
csv_file = "boundary.csv"  # Replace with your file path
df = pd.read_csv(csv_file)

# Convert to GeoDataFrame
# Assuming the columns for coordinates are named 'X' and 'Y'
geometry = [Point(xy) for xy in zip(df['X'], df['Y'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Set the Coordinate Reference System (CRS)
# Replace EPSG:4326 with your desired CRS
gdf.set_crs(epsg=32119, inplace=True)

# Save to Shapefile
shapefile_path = "output_geopandas.shp"
gdf.to_file(shapefile_path)

print(f"Shapefile saved to {shapefile_path}")


Shapefile saved to output_geopandas.shp


## Step 2: Retrieve data from ee

In [23]:
import ee
ee.Initialize(project='ee-tanvir54hossain')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [29]:
dem = ee.Image('USGS/3DEP/10m')
dem.getInfo()
print(dem.getInfo())

{'type': 'Image', 'bands': [{'id': 'elevation', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [3650412, 939612], 'crs': 'EPSG:4269', 'crs_transform': [9.259259259299957e-05, 0, -174.0005555570324, 0, -9.259259259299957e-05, 72.00055555584566]}], 'version': 1648043371709135, 'id': 'USGS/3DEP/10m', 'properties': {'system:footprint': {'type': 'LinearRing', 'coordinates': [[8.533243855035293, 72.00060200570647], [-36.52255593943564, 72.00060196804901], [-106.00421059777325, 72.00060205939565], [-174.0008093445371, 72.00060192652217], [-174.0006053017384, -15.000601990025386], [-134.22597532324247, -15.000602004951233], [-106.00421062932261, -15.000602024344811], [-63.754083308670964, -15.00060201818635], [-26.290103199499303, -15.000601982388343], [34.939573443906426, -15.000602040237249], [91.87822164793576, -15.000602000123632], [132.47795335015223, -15.000602038567253], [164.0006052984715, -15.000601961824241], [164.00080955172726, 72.0006018605616], [123.07069

In [31]:
import geemap
map = geemap.Map()
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [33]:
map.addLayer(dem)

In [38]:
geometrys = [ee.Geometry.Point([x,y], 'EPSG:32119') for x,y in zip(df['X'], df['Y'])]

In [39]:
fc = ee.FeatureCollection(geometrys)
map.add_layer(fc)

In [50]:
origin_info = fc.getInfo()
origin_info

{'type': 'FeatureCollection',
 'columns': {'system:index': 'String'},
 'features': [{'type': 'Feature',
   'geometry': {'crs': {'type': 'name', 'properties': {'name': 'EPSG:32119'}},
    'type': 'Point',
    'coordinates': [699102.8877924071, 186780.4458126684]},
   'id': '0',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'crs': {'type': 'name', 'properties': {'name': 'EPSG:32119'}},
    'type': 'Point',
    'coordinates': [699102.8877924071, 186780.4458126684]},
   'id': '1',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'crs': {'type': 'name', 'properties': {'name': 'EPSG:32119'}},
    'type': 'Point',
    'coordinates': [699105.8874190656, 186780.4458126684]},
   'id': '2',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'crs': {'type': 'name', 'properties': {'name': 'EPSG:32119'}},
    'type': 'Point',
    'coordinates': [699099.8881657487, 186777.44618600988]},
   'id': '3',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'crs': 

In [51]:
sampled_fc= dem.sampleRegions(
    collection=fc, 
    scale=10,    # Resolution of the image
    geometries=True)

In [52]:
sampled_info = sampled_fc.getInfo()
print(sampled_info)

{'type': 'FeatureCollection', 'columns': {}, 'properties': {'band_order': ['elevation']}, 'features': [{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [-78.01426489169957, 35.429736096570515]}, 'id': '0_0', 'properties': {'elevation': 22.24553871154785}}, {'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [-78.01426489169957, 35.429736096570515]}, 'id': '1_0', 'properties': {'elevation': 22.24553871154785}}, {'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [-78.01417506017115, 35.429736096570515]}, 'id': '2_0', 'properties': {'elevation': 22.477031707763672}}, {'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [-78.01426489169957, 35.429736096570515]}, 'id': '3_0', 'properties': {'elevation': 22.24553871154785}}, {'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [-78.01417506017115, 35.429736096570515]}, 'id': '4_0', 

In [56]:
for ind, itm in enumerate(origin_info['features']):
    itm['properties'] = sampled_info['features'][ind]['properties']

In [57]:
origin_info['features']

[{'type': 'Feature',
  'geometry': {'crs': {'type': 'name', 'properties': {'name': 'EPSG:32119'}},
   'type': 'Point',
   'coordinates': [699102.8877924071, 186780.4458126684]},
  'id': '0',
  'properties': {'elevation': 22.24553871154785}},
 {'type': 'Feature',
  'geometry': {'crs': {'type': 'name', 'properties': {'name': 'EPSG:32119'}},
   'type': 'Point',
   'coordinates': [699102.8877924071, 186780.4458126684]},
  'id': '1',
  'properties': {'elevation': 22.24553871154785}},
 {'type': 'Feature',
  'geometry': {'crs': {'type': 'name', 'properties': {'name': 'EPSG:32119'}},
   'type': 'Point',
   'coordinates': [699105.8874190656, 186780.4458126684]},
  'id': '2',
  'properties': {'elevation': 22.477031707763672}},
 {'type': 'Feature',
  'geometry': {'crs': {'type': 'name', 'properties': {'name': 'EPSG:32119'}},
   'type': 'Point',
   'coordinates': [699099.8881657487, 186777.44618600988]},
  'id': '3',
  'properties': {'elevation': 22.24553871154785}},
 {'type': 'Feature',
  'geomet

## create a feature class and add elevation data to the features

In [70]:
import os
fcname = os.path.join(arcpy.env.workspace,'pnt_elev.shp')
if arcpy.Exists(fcname):
    arcpy.management.Delete(fcname)
arcpy.management.CreateFeatureclass(arcpy.env.workspace, 'pnt_elev.shp',geometry_type='POINT', spatial_reference=32119)

<Result 'D:\\project2\\pnt_elev.shp'>

In [71]:
import arcpy
arcpy.management.AddField(fcname, field_name='elevation', field_type='FLOAT')

<Result 'D:\\project2\\pnt_elev.shp'>

In [73]:
with arcpy.da.InsertCursor(fcname, ['SHAPE@', 'elevation']) as cursor:
    for feat in origin_info['features']:
        # get the coordinates and create a pointgeometry
        coords = feat['geometry']['coordinates']
        pnt = arcpy.PointGeometry(arcpy.Point(coords[0], coords[1]), spatial_reference=32119)
        # get the properties and write it to elevation
        elev = feat['properties']['elevation']
        cursor.insertRow([pnt, elev])